In [8]:
import torch
import math
for method in ['mle', 'dru', 'drw']:
    for mis in ['bth', 'opc', 'psc', 'bad']:
        for estimator in ['LATE', 'MLATE']:
            result = torch.load('{}_{}_{}.pt'.format(method, mis, estimator))
            bias = torch.abs(result.mean(0) - torch.tensor([0.0,-1.0])) * 100
            std = torch.sqrt(torch.mean((result - torch.tensor([0.0,-1.0]))**2, 0)) / math.sqrt(result.shape[0]) * 100
            print('{}_{}_{}.pt'.format(method, mis, estimator), bias.tolist(), std.tolist())

mle_bth_LATE.pt [4.5534586906433105, 9.30579948425293] [0.43561193346977234, 0.9288403987884521]
mle_bth_MLATE.pt [13.811415672302246, 18.330276489257812] [0.8098511099815369, 1.3241429328918457]
mle_opc_LATE.pt [5.856235504150391, 11.942678451538086] [0.4799013137817383, 0.947130560874939]
mle_opc_MLATE.pt [16.469945907592773, 21.107900619506836] [0.8535577058792114, 1.3465123176574707]
mle_psc_LATE.pt [5.856235504150391, 11.942678451538086] [0.4799013137817383, 0.947130560874939]
mle_psc_MLATE.pt [16.469945907592773, 21.107900619506836] [0.8535577058792114, 1.3465123176574707]
mle_bad_LATE.pt [19.145633697509766, 2.7969956398010254] [0.6896345019340515, 0.6246792078018188]
mle_bad_MLATE.pt [41.27730178833008, 0.5283236503601074] [1.4587193727493286, 1.0534470081329346]
dru_bth_LATE.pt [0.9537761211395264, 6.1550140380859375] [0.4465446472167969, 1.0355411767959595]
dru_bth_MLATE.pt [3.5834755897521973, 9.746253967285156] [0.9997743964195251, 1.7903261184692383]
dru_opc_LATE.pt [1.208